In [36]:
import numpy as np
from matplotlib import pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
cd /content/gdrive/My Drive/PL project

/content/gdrive/My Drive/PL project


In [38]:
ls 

My_noisy_H_12.mat  My_noisy_H_22.mat  Perfect_H_40000.mat


In [39]:
import numpy as np
import math
from scipy.io import loadmat # to load .mat file
import matplotlib.pyplot as plt # to plot figure
from scipy import interpolate 

In [40]:
def interpolation(noisy , SNR , Number_of_pilot , interp):
    noisy_image = np.zeros((len(noisy),72,14,2))

    noisy_image[:,:,:,0] = np.real(noisy) 

    noisy_image[:,:,:,1] = np.imag(noisy)


    if (Number_of_pilot == 48):
        idx = [14*i for i in range(1, 72,6)]+[4+14*(i) for i in range(4, 72,6)]+[7+14*(i) for i in range(1, 72,6)]+[11+14*(i) for i in range(4, 72,6)]
    elif (Number_of_pilot == 16):
        idx= [4+14*(i) for i in range(1, 72,9)]+[9+14*(i) for i in range(4, 72,9)]
    elif (Number_of_pilot == 24):
        idx = [14*i for i in range(1,72,9)]+ [6+14*i for i in range(4,72,9)]+ [11+14*i for i in range(1,72,9)]
    elif (Number_of_pilot == 8):
      idx = [4+14*(i) for  i in range(5,72,18)]+[9+14*(i) for i in range(8,72,18)]
    elif (Number_of_pilot == 36):
      idx = [14*(i) for  i in range(1,72,6)]+[6+14*(i) for i in range(4,72,6)] + [11+14*i for i in range(1,72,6)]



    r = [x//14 for x in idx]
    c = [x%14 for x in idx]



    interp_noisy = np.zeros((len(noisy),72,14,2))

    for i in range(len(noisy)):
        z = [noisy_image[i,j,k,0] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,0] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,0] = z_intp
        z = [noisy_image[i,j,k,1] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,1] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,1] = z_intp


    interp_noisy = np.concatenate((interp_noisy[:,:,:,0], interp_noisy[:,:,:,1]), axis=0).reshape(len(noisy)*2, 72, 14, 1)
   
    
    return interp_noisy
   

In [42]:
    idx = [14*i for i in range(1, 72,6)]+[4+14*(i) for i in range(4, 72,6)]+[7+14*(i) for i in range(1, 72,6)]+[11+14*(i) for i in range(4, 72,6)]
    r = [x//14 for x in idx]#indexes of pilot locations
    c = [x%14 for x in idx]
    #f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float),z,function='gaussian')
    for j,k in zip(r,c):
      print(j,k)
    print(idx)

1 0
7 0
13 0
19 0
25 0
31 0
37 0
43 0
49 0
55 0
61 0
67 0
4 4
10 4
16 4
22 4
28 4
34 4
40 4
46 4
52 4
58 4
64 4
70 4
1 7
7 7
13 7
19 7
25 7
31 7
37 7
43 7
49 7
55 7
61 7
67 7
4 11
10 11
16 11
22 11
28 11
34 11
40 11
46 11
52 11
58 11
64 11
70 11
[14, 98, 182, 266, 350, 434, 518, 602, 686, 770, 854, 938, 60, 144, 228, 312, 396, 480, 564, 648, 732, 816, 900, 984, 21, 105, 189, 273, 357, 441, 525, 609, 693, 777, 861, 945, 67, 151, 235, 319, 403, 487, 571, 655, 739, 823, 907, 991]


In [43]:
channel_model = "VehA"
SNR = 12
Number_of_pilots = 48
perfect = loadmat("Perfect_H_40000.mat")['My_perfect_H']
noisy_input = loadmat("My_noisy_H_12.mat")['My_noisy_H']

print(perfect.shape)
print(noisy_input.shape)

(40000, 72, 14)
(40000, 72, 14)


In [44]:
noisy_input = noisy_input[0:1000]
perfect = perfect[0:1000]

print(perfect.shape)
print(noisy_input.shape)

(1000, 72, 14)
(1000, 72, 14)


In [45]:
interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')
print(SNR,Number_of_pilots)

12 48


In [46]:
interp_noisy.shape

(2000, 72, 14, 1)

In [47]:
perfect_image = np.zeros((len(perfect),72,14,2))
perfect_image[:,:,:,0] = np.real(perfect)
perfect_image[:,:,:,1] = np.imag(perfect)
perfect_image = np.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)

In [48]:
perfect_image.shape

(2000, 72, 14, 1)

In [49]:
idx_random = np.random.rand(len(perfect_image)) < (1/9)  
train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]    

print(train_data.shape)
print(val_data.shape)

(234, 72, 14, 1)
(1766, 72, 14, 1)


In [50]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, BatchNormalization, Activation, Subtract
from tensorflow.keras.models import Sequential,  Model
from tensorflow.keras.callbacks import ModelCheckpoint

def SRCNN_model():

    input_shape = (72,14,1)
    x = Input(shape = input_shape)
    c1 = Conv2D( 64 , 9, activation = 'relu', padding='same')(x)
    c2 = Conv2D( 32 , 1, activation = 'relu', padding='same')(c1)
    c3 = Conv2D( 1 , 1,padding='same')(c2)
    model = Model(inputs = x, outputs = c3)
    ##compile
    #adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error']) 
    return model
def DNCNN_model ():
  
    inpt = Input(shape=(None,None,1))
    # 1st layer, Conv+relu
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 18 layers, Conv+BN+relu
    for i in range(18):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    #adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])    
    return model


In [51]:

sr_model = SRCNN_model()
sr_model.summary()
history1 = sr_model.fit(train_data, train_label, validation_data=(val_data, val_label), shuffle=True, epochs= 100)


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 72, 14, 1)]       0         
                                                                 
 conv2d_43 (Conv2D)          (None, 72, 14, 64)        5248      
                                                                 
 conv2d_44 (Conv2D)          (None, 72, 14, 32)        2080      
                                                                 
 conv2d_45 (Conv2D)          (None, 72, 14, 1)         33        
                                                                 
Total params: 7,361
Trainable params: 7,361
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 3s 317ms/step - loss: 0.2311 - mean_squared_error: 0.2311 - val_loss: 0.0551 - val_mean_squared_error: 0.0551
Epoch 2/100
8/8 [===========

In [53]:
print(history1.history['loss'])

[0.23106609284877777, 0.05931198596954346, 0.035894203931093216, 0.0284905843436718, 0.025232894346117973, 0.022615700960159302, 0.020995095372200012, 0.019474944099783897, 0.018382292240858078, 0.017645204439759254, 0.017086250707507133, 0.01669471710920334, 0.01622830145061016, 0.015925979241728783, 0.015651527792215347, 0.015529354102909565, 0.01519276387989521, 0.015129589475691319, 0.014956657774746418, 0.014776499941945076, 0.014704613946378231, 0.014485684223473072, 0.014360902830958366, 0.014317533932626247, 0.01415882259607315, 0.014021539129316807, 0.013837924227118492, 0.01373809203505516, 0.013645825907588005, 0.013516254723072052, 0.013511976227164268, 0.01338986773043871, 0.013301785103976727, 0.013228168711066246, 0.013295874930918217, 0.0131228594109416, 0.013093480840325356, 0.013033303432166576, 0.0129955243319273, 0.013020035810768604, 0.012899248860776424, 0.012764782644808292, 0.012771835550665855, 0.012676866725087166, 0.012695123441517353, 0.012632345780730247, 0

In [55]:
sr_pred_train = sr_model.predict(train_data)
sr_pred_validation = sr_model.predict(val_data)

print(sr_pred_train.shape)
print(sr_pred_validation.shape)

56/56 [==============================] - 2s 29ms/step
(234, 72, 14, 1)
(1766, 72, 14, 1)


In [ ]:
dn_model = DNCNN_model()
dn_model.summary()
dn_model.fit(train_data, train_label, validation_data=(val_data, val_label),shuffle=True, epochs= 100)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d_66 (Conv2D)             (None, None, None,   640         ['input_6[0][0]']                
                                64)                                                               
                                                                                                  
 activation_57 (Activation)     (None, None, None,   0           ['conv2d_66[0][0]']              
                                64)                                                         